In [1]:
import numpy as np


In [2]:
from sklearn.metrics import fbeta_score


In [9]:
LABEL_TRESHOLD = 0.2

def f2_score(y_true, y_pred):
    return fbeta_score(y_true, y_pred > LABEL_THRESHOLD, beta=2, average='samples')

In [8]:
true_target = np.loadtxt('true_target.csv')

In [5]:
predictions = np.loadtxt('predictions.csv')

In [65]:
filter_array = np.array([0.01,0.02,0.03,0.04,0.05,0.06,0.07,
                        0.08,0.09,0.10,0.11,0.12,0.13,0.14,
                        0.15,0.16,0.17])

In [14]:
example = np.array([[0.10,0.10,0.10,0.10,0.10,0.10,0.10,
                        0.10,0.10,0.10,0.10,0.10,0.10,0.10,
                        0.10,0.10,0.10],
                   [0.05,0.05,0.05,0.05,0.05,0.05,0.05,
                        0.05,0.05,0.05,0.05,0.05,0.05,0.05,
                        0.05,0.05,0.05],
                   [0.15,0.15,0.15,0.15,0.15,0.15,0.15,
                        0.15,0.15,0.15,0.15,0.15,0.15,0.15,
                        0.15,0.15,0.15]
                   ])

In [15]:
example > filter_array

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False, False, False, False, False, False, False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False, False, False]], dtype=bool)

In [16]:
true_target

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
def f_opt(threshold):
    return - fbeta_score(true_target, predictions > threshold, beta=2, average='samples')

In [25]:
f_opt(filter_array)

-0.82662824639357735

In [26]:
import scipy.optimize as optimize

In [27]:
xopt = optimize.fmin(f_opt, filter_array, xtol=1e-8, disp=True)

Optimization terminated successfully.
         Current function value: -0.842704
         Iterations: 640
         Function evaluations: 1144


In [28]:
xopt

array([ 0.02731559,  0.0271005 ,  0.02662923,  0.07731717,  0.0342785 ,
        0.10929462,  0.04310637,  0.04655365,  0.1809137 ,  0.06826109,
        0.11079291,  0.07854928,  0.02996431,  0.18858926,  0.13480435,
        0.02359167,  0.09036682])

In [69]:
filter_array = np.array([-2,30,4,0.0004,0.0895,1.5,89,
                        1000,-35,22,1,0,-0.0987,0.14,
                        0.15,54780,-170])

In [49]:
xopt = optimize.fmin(f_opt, filter_array, xtol=1e-8, disp=True)

Optimization terminated successfully.
         Current function value: -0.508790
         Iterations: 263
         Function evaluations: 999


In [31]:
xopt

array([ -2.49557834e+00,   3.29638128e+01,   4.72411970e+00,
         4.40643107e-04,   9.10924049e-02,   1.84315810e+00,
         8.96740927e+01,   8.76809338e+02,  -3.01608331e+01,
         2.10540110e+01,   9.95828035e-02,   4.40194090e-04,
        -9.99283099e-02,   1.50942106e-01,   1.52730232e-01,
         5.77222811e+04,  -1.50275866e+02])

In [32]:
constraints = [(0,1) for i in range(17)]

In [33]:
constraints

[(0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1)]

In [54]:
x1opt = optimize.fmin_l_bfgs_b(f_opt, filter_array, approx_grad = True, bounds=constraints, pgtol=1e-8)

In [55]:
x1opt

(array([ 0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,
         0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17]),
 -0.82662824639357735,
 {'funcalls': 18,
  'grad': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.]),
  'nit': 0,
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0})

In [61]:
# Cobyla constraints are build by comparing return value with 0.
# They must be >= 0 or be rejected

def constr_sup0(x):
    return np.min(x)
def constr_inf1(x):
    return 1 - np.max(x)

In [70]:
x2opt = optimize.fmin_cobyla(f_opt, filter_array, [constr_sup0,constr_inf1])

COBYLA failed to find a solution: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.


In [67]:
x2opt

array([ 0.26280837,  0.96995811,  0.99543876,  0.67959601,  0.02391652,
        0.43268884,  0.15414837,  0.02334634,  0.38875806,  0.11339665,
        0.19358153,  0.09235863,  0.17984985,  0.35286687,  0.21257828,
        0.19223617,  0.12633189])

In [68]:
f_opt(x2opt)

-0.87837138210781296

In [71]:
filter_array = np.array([0.02,0.02,0.02,0.02,0.02,0.02,0.02,
                        0.02,0.02,0.02,0.02,0.02,0.02,0.02,
                        0.02,0.02,0.02])

In [72]:
x2opt = optimize.fmin_cobyla(f_opt, filter_array, [constr_sup0,constr_inf1])

/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [73]:
x2opt

array([ 0.29695534,  0.61885414,  0.95819905,  0.9245769 ,  0.96039942,
        0.40446729,  0.2360938 ,  0.92673439,  0.27731897,  0.10289876,
        0.17834395,  0.08832439,  0.1517644 ,  0.21374738,  0.97413643,
        0.5231204 ,  0.12911092])

In [74]:
f_opt(x2opt)

-0.87940905151433246

In [77]:
thr_0 = np.array([0.2 for i in range(17)])

In [78]:
thr_0

array([ 0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,  0.2,
        0.2,  0.2,  0.2,  0.2,  0.2,  0.2])